# Track sample-level metadata

In [ ]:
!lamin delete test-link-samples
!lamin init --storage ./test-link-samples --schema bionty

We already saw how to link data objects to entities representing features during ingestion.

For sample-level metadata, the underlying schema is often more complicated, and hence it's best done in a separate step.

Here, we walk through this process.

In [ ]:
import lamindb as ln
import lnschema_bionty as lb

ln.track()

Samples, i.e., metadata associated with observations, are linked with the same approach post-ingestion.

In [ ]:
ln.File(ln.dev.datasets.file_mini_csv()).save()

Let's first query an RNA-seq dataset stored as an `.csv` file.

In [ ]:
file = ln.select(ln.File, suffix=".csv").first()

In [ ]:
file

For instance, let's annotate a scRNA-seq dataset with its readout type (scRNA-seq), the tissue, and the species.

## Readout

In [ ]:
readout_bionty = lb.Readout.bionty()  # equals to bionty.Readout()
readout_bionty_lookup = readout_bionty.lookup()

In [ ]:
readout_bionty_lookup.single_cell_RNA_sequencing

In [ ]:
readout_record = lb.Readout.from_bionty(
    readout_bionty_lookup.single_cell_RNA_sequencing
)

ln.save(readout_record)

### CellType

In [ ]:
celltype_bionty = lb.CellType.bionty()  # equals to bionty.CellType()
celltype_bionty_lookup = celltype_bionty.lookup()

In [ ]:
celltype_bionty_lookup.CD8_positive_alpha_beta_memory_T_cell

In [ ]:
celltype_record = lb.CellType.from_bionty(
    celltype_bionty_lookup.CD8_positive_alpha_beta_memory_T_cell
)

In [ ]:
celltype_record.save()

## Link against file

Link metadata records against the data object:

In [ ]:
file.readouts.add(readout_record)

In [ ]:
file.cell_types.add(celltype_record)

## Query file from linked metadata

In [ ]:
ln.File.select(readouts__name__icontains="single-cell").df()

In [ ]:
ln.File.select(cell_types__name__contains="T cell").df()

In [ ]:
!lamin delete test-link-samples
!rm -r ./test-link-samples